In [17]:
import pandas as pd

from sklearn.impute import SimpleImputer
from sklearn.preprocessing import StandardScaler
from sklearn.preprocessing import OrdinalEncoder
from sklearn.pipeline import Pipeline
from sklearn.compose import ColumnTransformer


In [3]:
df= pd.DataFrame(pd.read_csv("./data/gemstone.csv"))
df.head()

,id,carat,cut,color,clarity,depth,table,x,y,z,price
0,0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453


In [4]:
df.drop(columns="id",axis=1,inplace=True)

In [5]:
df

,carat,cut,color,clarity,depth,table,x,y,z,price
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55,13619
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05,13387
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50,2772
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71,666
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77,14453
...,...,...,...,...,...,...,...,...,...,...
193568,0.31,Ideal,D,VVS2,61.1,56.0,4.35,4.39,2.67,1130
193569,0.70,Premium,G,VVS2,60.3,58.0,5.75,5.77,3.47,2874
193570,0.73,Very Good,F,SI1,63.1,57.0,5.72,5.75,3.62,3036
193571,0.34,Very Good,D,SI1,62.9,55.0,4.45,4.49,2.81,681


In [6]:
X=df.drop(labels="price",axis=1)
X.head()

,carat,cut,color,clarity,depth,table,x,y,z
0,1.52,Premium,F,VS2,62.2,58.0,7.27,7.33,4.55
1,2.03,Very Good,J,SI2,62.0,58.0,8.06,8.12,5.05
2,0.70,Ideal,G,VS1,61.2,57.0,5.69,5.73,3.50
3,0.32,Ideal,G,VS1,61.6,56.0,4.38,4.41,2.71
4,1.70,Premium,G,VS2,62.6,59.0,7.65,7.61,4.77


In [9]:
Y=df['price']
Y.head()

0    13619
1    13387
2     2772
3      666
4    14453
Name: price, dtype: int64

### Dividing the Groups into two Groups

In [10]:
categorical_columns = X.select_dtypes(include = 'object').columns
numerical_columns = X.select_dtypes(exclude= 'object').columns

### Creating the Categories

In [11]:
cut_categories = ['Fair', 'Good', 'Very Good',  'Premium', 'Ideal']
color_categories = ['D', 'E', 'F', 'G', 'H', 'I', 'J']
clarity_categories = ["I1", 'SI2', 'SI1', 'VS2', 'VS1', 'VVS2', 'VVS1', 'IF']

### Creating Different Pipelines

In [18]:
num_pipe = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy= 'median')), 
        ('scaler', StandardScaler())
    ]
)

cat_pipe = Pipeline(
    steps = [
        ('imputer', SimpleImputer(strategy='most_frequent')), 
        ('ordinalencoder', OrdinalEncoder(categories=[cut_categories, color_categories, clarity_categories])), 
        ('scaler', StandardScaler())
    ]

)

In [19]:
preprocessor = ColumnTransformer([
    ('num_pipe', num_pipe, numerical_columns), 
    ('cat_pipe', cat_pipe, categorical_columns)
])

### Splitting the Data

In [20]:
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X, Y, test_size= 0.25, random_state= 45)

In [21]:
X_train = pd.DataFrame(preprocessor.fit_transform(X_train), columns=preprocessor.get_feature_names_out()) 
X_test = pd.DataFrame(preprocessor.transform(X_test), columns = preprocessor.get_feature_names_out())

In [22]:
X_train.head()

,num_pipe__carat,num_pipe__depth,num_pipe__table,num_pipe__x,num_pipe__y,num_pipe__z,cat_pipe__cut,cat_pipe__color,cat_pipe__clarity
0,-0.195410,0.721990,-0.640704,-0.049355,-0.081201,0.008907,0.873256,-0.317345,-1.315751
1,-0.173766,-0.203669,0.401246,-0.004238,0.018691,-0.005603,-0.133146,2.145303,0.017304
2,1.579392,0.721990,0.922221,1.457564,1.444428,1.518017,-0.133146,0.913979,-1.315751
3,3.267619,-0.481366,-1.161679,2.567452,2.516001,2.446701,0.873256,2.145303,0.017304
4,-0.455137,-0.111103,-0.640704,-0.347130,-0.308229,-0.324838,0.873256,1.529641,2.683414


In [23]:
X_test.head()

,num_pipe__carat,num_pipe__depth,num_pipe__table,num_pipe__x,num_pipe__y,num_pipe__z,cat_pipe__cut,cat_pipe__color,cat_pipe__clarity
0,0.475552,-0.759064,-1.161679,0.708616,0.736100,0.632866,0.873256,-0.317345,0.017304
1,-1.191031,-0.666498,0.922221,-1.565299,-1.552344,-1.587267,-1.139547,-1.548669,1.350359
2,-0.996236,-0.018537,-1.161679,-1.168266,-1.143693,-1.151947,-1.139547,0.913979,1.350359
3,-0.498425,-0.759064,1.443196,-0.356154,-0.389959,-0.426413,-0.133146,-0.317345,0.017304
4,0.237469,-3.443474,0.922221,0.474006,0.445504,0.139503,-0.133146,-1.548669,-0.649224


In [24]:
from sklearn.linear_model import LinearRegression, Lasso, Ridge, ElasticNet
from sklearn.metrics import r2_score, root_mean_squared_error, mean_absolute_error, mean_squared_error
import numpy as np

In [28]:

def model_evaluation(true, predicted):
    mae = mean_absolute_error(true, predicted)
    mse = mean_squared_error(true, predicted)
    rmse = np.sqrt(mean_squared_error(true, predicted))
    r2_square = r2_score(true, predicted)
    return mae, rmse, r2_square

In [29]:
models={
    'LinearRegression':LinearRegression(),
    'Lasso':Lasso(),
    'Ridge':Ridge(),
    'Elasticnet':ElasticNet()
}
model_list=[]
r2_list=[]

for i in range(len(list(models))):
    model=list(models.values())[i]
    model.fit(X_train,y_train)

    #Make Predictions
    y_pred=model.predict(X_test)

    mae, rmse, r2_square=model_evaluation(y_test,y_pred)

    print(list(models.keys())[i])
    model_list.append(list(models.keys())[i])
    

    print('Model Training Performance')
    print("RMSE:",rmse)
    print("MAE:",mae)
    print("R2 score",r2_square*100)

    r2_list.append(r2_square)
    
    print('*'*35)
    print('\n')

LinearRegression
Model Training Performance
RMSE: 1015.9917605432947
MAE: 679.705409558577
R2 score 93.71745654465842
***********************************


Lasso
Model Training Performance
RMSE: 1016.1357781042951
MAE: 680.8409748615863
R2 score 93.71567530833693
***********************************


Ridge
Model Training Performance
RMSE: 1015.9949164269083
MAE: 679.7333205816674
R2 score 93.71741751480107
***********************************


Elasticnet
Model Training Performance
RMSE: 1540.009206276005
MAE: 1065.2324249765468
R2 score 85.565499379458
***********************************




In [30]:
model_list

['LinearRegression', 'Lasso', 'Ridge', 'Elasticnet']